<a href="https://colab.research.google.com/github/jleandroforte/Segunda-Entrega-Forte/blob/main/Segunda_pre_entrega_Forte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Introduccion**

El dataset con el que vamos a trabajar recopila datos de ventas y actividad de los clientes en el sitio web de una farmacia online a lo largo de 90 días.

Tenemos información sobre varias características de los productos que serán detalladas en secciones siguientes, sus precios, los precios de la competencia y comportamiento de los clientes, si hacen click en un producto, si los colocan en una canasta de productos y finalmente si compran un producto. Nótese que no todas las líneas representan ventas.

La clave del dataset es que la farmacia sigue una política de 'pricing dinámico' donde los precios de cada producto son ajustados diariamente, dentro de ciertas bandas.

#**Objetivos**

Nuestro objetivo va a ser predecir el revenue, y para ello vamos a utilizar 2 enfoques:

**Modelo de Regresion**

basandonos en lo siguiente:

> 1. Nuestros datos están etiquetados. En principio, no tenemos la necesidad de hacer un análisis no supervisado.

> 2. La variable independiente es el revenue, o ingresos que generan las ventas (no todas las líneas representan ventas, por tanto, el revenue es 0 en muchas líneas), es una variable numérica continua, es decir, no son dicótomicas como podría requerir una regresión logística, con lo cual un modelo de regresión, en principio, se adecúa a nuestros objetivos. Recordemos que el objetivo al final del curso es usar este dataset para construir un modelo que **prediga** el revenue en base a las restantes variables donde, fundamentalmente, lo que cambia es el vector de precios.

**Modelo de clasificación $decision$ $tree$**

En este caso, no vamos a predecir directamente el revenue, sino la variable order, que, recordemos, toma valores 0 y 1, cuando es 1, significa que una línea representa una venta, y esa venta tiene un revenue asociado. Es decir, corremos ligeramente el foco del análisis, y como paso intermedio, exploramos **que variables determinan las ventas**


In [ ]:
# Importamos las liberías necesarias.
from google.colab import drive
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import metrics


drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Entregas/DB'

In [ ]:
# importamos los archivos y los combinamos en un solo dataset llamado 'farmacia' a través de la variable product id.

pricing_class=pd.read_csv("class.csv", sep='|')
pricing_items=pd.read_csv('items.csv', sep='|')
pricing_train=pd.read_csv('train.csv', sep='|')

dataset = pd.merge(pricing_train, pricing_items, on='pid');
farmacia=pd.DataFrame(dataset)

In [ ]:
farmacia

In [ ]:
farmacia.shape

In [ ]:
farmacia.info

In [ ]:
farmacia.describe(include='all')

In [ ]:
farmacia.dtypes

#**Descripción de las variables:**

**Vamos a proceder a describir verbalmente nuestras variables, cuando se trate de variables que no varían en el tiempo, se explicita en la descripción**

**day**: el día que se registra, recordemos que tenemos datos 91 días de ventas y otras acciones de usuarios en el sitio web. El datatype es int.

**pid**: El id del producto, como vimos, tenemos más de 22 mil productos. El datatype es int, los productos se identifican por un numero, no por su nombre. Es una variable invariante.

**adFlag**: Nos indica si el producto en cuestión es objeto de una campaña publicitaria. El valor 1 indica que hubo capaña, y 0 indica que no la hubo.

**availability**: Status de disponibilidad de los productos, toma los valores {1,2,3,4}

**competitorPrice** : El precio de la competencia para un producto. Es un floating point.

**click, basket, order**: Denotan acciones de los usuarios, si hicieron click en un producto, si lo colocaron en un carrito de compra (pero no necesariamente lo compraron), mientras que order indica si efectivamente el registro denota una compra. Las 4 variables toman los valores {0,1}, donde 1 denota acción (compra, carrito de compra, click) y el 0 la ausencia de acción.

**price**: El precio efectivamente asociado a la observacion. Recordemos que la farmacia sigue una estrategia de 'pricing dinámico', los precios de cada producto se ajustan día a día. También es importante tener en cuenta que no se trata de precios customizados para clientes, los ajustes de precios son diarios y automáticos. Esto introduce una gran variabilidad en las observaciones que nos permitirá evaluar la influencia de los precios en las ventas, y es el insumo fundamental para predecir la demanda futura en función de como ajustemos nuestros precios. El precio es un floating point.

**revenue**: Los ingresos por ventas. Es decir, el precio multiplicado por las cantidades compradas (que no observamos, solo observamos precios, compras (la variable 'order') y revenue). El revenue es un floating point.

**manufacturer**: El fabricante de cada producto. Se identifica mediante un numero entero. Es invariante, al igual que el product id.

**group**: El grupo de productos, combina letras y numeros. Es invariante.

**content**: El contenido de un producto, se identifica bajo la nomenclatura numeroXnumero, por caso: 5X10. Es invariante.

**unit**: La unidad del producto, es un string de mayúsculas.Es invariante.

**pharmForm**: La dosis: son 3 letras mayúsculas. Es invariante.

**genericProduct**: Si se trata de un medicamento generico, toma los valores {0,1}, el 1 indica que se trata de un medicamento genérico. Es invariante.

**salesIndex**: un código de dispensión de medicamentos de Estados unidos. Es un entero. Es invariante.

**category**: categoría de negocio: es un numero de negocio. Es invariante.

**campaignIndex**: Tipo de campaña publicitaria de que fue objeto el producto, toma los valores {A,B,C}

**rrp**: El precio de referencia, recordemos que la farmacia ajusta los precios de cada producto diariamente, pero cada item tiene un precio de referencia, más adelante vamos a graficar algunos ejemplos. Es invariante, la variabilidad está en los precios efectivos.

#**Módulo de Limpieza de datos**

#**Tratamiento de Missing Values**

En primer lugar, vemos cuantos datos nos faltan por columna, y proponemos las siguientes soluciones para aquellas variables donde identificamos missing values:

Para competitor price vamos a reemplazar los missing values por la mediana del precio por cada producto, no en general, en caso de no disponer de un precio para algún producto, se reemplaza por la mediana de la variable en el dataset completo.

para pharmForm, al menos por ahora, vamos a reemplazar por el valor más frecuente, a nivel de producto y no en el agregado. No se pueden usar medidas como media o mediana porque no es una variable numérica.

Para category usamos el mismo procedimiento que para pharmForm, con las mismas consideraciones.

Para campaing index, que en el dataset toma los valores {A,B,C}, e indica el tipo de campaña publicitaria que se llevó a cabo, tenemos 2 tipos de tratamiento, dado que es dependiente de adFlag, solo cuando adFlag es igual a 1 hay campaña publicitaria, de modo que si adFlag==0, reemplazamos por "D", que es una manera de indicar que no hay campaña publicitaria, mientras que para los casos en que adFlag==1 reemplazamos por la moda o valor más repetido a nivel de producto.

In [ ]:
datos_faltantes = farmacia.isnull().sum()
print("Datos faltantes por columna: " , datos_faltantes)

Reemplazo de missing values para la variable competitorPrice, si a nivel de producto no hay una mediana, se reemplaza por la mediana de la columna en general.

In [ ]:
medianas_faltantes = farmacia.groupby('pid')['competitorPrice'].transform('median') # recordemos que pid es 'Product Id', por eso hacemos el reemplazo a ese nivel.

farmacia['competitorPrice'] = farmacia['competitorPrice'].fillna(medianas_faltantes)

# En los casos no capturados por las lineas anteriores reemplazamos por la mediana general de la variable:
mediana_competitorPrice = farmacia['competitorPrice'].median()

farmacia['competitorPrice'] = farmacia['competitorPrice'].fillna(mediana_competitorPrice)

Reemplazo de missing values para las variables pharmForm y category, estamos usando el valor más frecuente, por eso la función hace referencia a la moda ("mode")

In [ ]:
def completar_pharmform(series):
    if series.mode().empty:
        return series
    else:
        moda_pharmForm = series.mode().iloc[0]
        return series.fillna(moda_pharmForm)

farmacia['pharmForm'] = farmacia.groupby('pid')['pharmForm'].transform(completar_pharmform)

farmacia['pharmForm'].fillna('default_value', inplace=True) # esta ultima linea de codigo es para reemplazar missing values en casos no capturados por la funcion anterior.

In [ ]:
def completar_category(series):
    if series.mode().empty:
        return series
    else:
        moda_category = series.mode().iloc[0]
        return series.fillna(moda_category)

farmacia['category'] = farmacia.groupby('pid')['category'].transform(completar_category)

farmacia['category'].fillna('default_value', inplace=True) # esta ultima linea de codigo es para reemplazar missing values en casos no capturados por la funcion anterior.



Reemplazo de missing values para campaignIndex, recordemos el metodo:
tenemos 2 tipos de tratamiento, dado que es dependiente de adFlag, solo cuando adFlag es igual a 1 hay campaña publicitaria, de modo que si adFlag==0, reemplazamos por "D" (ya que las campañas son {A,B,C}, y de esta forma con la "D" podemos identificar rápidamente que no hay campaña) para los casos en que adFlag==1 reemplazamos por la moda o valor más repetido a nivel de producto.  

In [ ]:
farmacia['campaignIndex'].fillna('D', inplace=True)

mask = (farmacia['adFlag'] == 1) & (farmacia['campaignIndex'].isna())
farmacia.loc[mask, 'campaignIndex'] = farmacia[mask].groupby('pid')['campaignIndex'].transform(lambda x: x.mode().iloc[0])

farmacia['campaignIndex'].fillna('default_value', inplace=True) # esta ultima linea de codigo es para reemplazar missing values en casos no capturados por la funcion anterior.


Ahora constatamos que ya no tenemos más missing values:



In [ ]:
datos_faltantes = farmacia.isnull().sum()
print("Datos faltantes por columna: " , datos_faltantes)


#**Modelo de regresión lineal**

##**Selección de variables para incluir como regresores**

Usamos forward Selection

In [ ]:
!pip install mlxtend --upgrade # instalamos mlxtend

En esta sección generamos nuestras variable dependiente, y=revenue, y excluimos
del análisis las variables no numéricas que no pudimos transformar mediante el encoding en la sección previa.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [ ]:
X = farmacia.drop([ 'revenue' , 'group', 'unit', 'pharmForm', 'campaignIndex' , 'content', 'category', 'day'  ], axis=1)
y = farmacia['revenue']

In [ ]:
print(X.dtypes) #volvemos a chequear que nuestros regresores son variables numericas aptas para implementar un modelo OLS.print(X.dtypes) #volvemos a chequear que nuestros regresores son variables numericas aptas para implementar un modelo OLS.

##**División del Dataset en test y training sets**

Establecemos que la fracción del dataset que usamos para testear es del 20%, tenemos un dataset grande, con 2,7 millones de observaciones, de modo que separar el 20% de las observaciones deber+ía ser suficiente para entrenar y testear modelos.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

##**Forward Selection**

:La métrica a evaluar para determinar las variables relevantes es el r cuadrado de la regresión:

In [ ]:
#hacemos forward selection
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
# Sequential Forward Selection(sfs)
sfs = SFS(LinearRegression(),
          k_features=11,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)

In [ ]:
sfs = sfs.fit(X_train, y_train)
sfs.k_feature_names_

Vemos que nos quedamos con los siguientes regresores, que ya fueron explicados y analizados en detalle en secciones anteriores,

 > 'pid': Product ID

 > 'adFlag': Campaña de publicidad

 > 'competitorPrice' : Precio de la competencia

 > 'click' : Si los usuarios hacen click en un producto.

 > 'order' : Si los usuarios compran un producto

 > 'price': Precio efectivo de venta

 > 'genericProduct': Si el producto es un medicamento genérico

 > 'salesIndex': un código de dispensión de medicamentos de Estados unidos.

 > 'rrp' : Precios de referencia

 LineId no es un regresor a pesar del resultado del SFS porque es simplemente el orden en el que vienen los datos y no responde a ninguna caracteristica de los productos.

##**Estimacion del modelo OLS**

Y finalmente estimamos un modelo simple de OLS donde tratamos de predecir el revenue en función de los regresores que nos arroja el modelo de forward selection implementado más arriba.


Como paso previo regeneramos nuesto vector de variables independientes ("X") para que solo incluya los regresores que surjen como resultado del Forward Selection

In [ ]:
X = farmacia.drop([ 'revenue' , 'basket', 'campaignIndex', 'category', 'content' , 'day' , 'group', 'lineID' , 'manufacturer' , 'unit' , 'pharmForm'  ], axis=1)
y = farmacia['revenue']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
import statsmodels.api as sm

modelo_ols = sm.OLS(y_train, X_train).fit()
print(model.summary())

Como vemos, el r2 es de solo 0.528. Como se muestra en el siguiente apartado, las correlaciones de los regresores con el revenue es relativametne débil, y, en mi opinión, por ahí pasa la clave por las poca efectividad del modelo.

También vamos a tener que estudiar la multicolinealidad de los regresores, como indica la propia salida del modelo.

Deberemos estudiar modelos no lineales u otros algoritmos de regresión más sofisticados para mejorar nuestras predicciones.

Otras observaciones que surgen del resultado de la regresión lineal son,

Los p-values son bajos, pero deben entenderse en el contexto de un modelo no muy potente.

> El hecho de que la covariance Type esté indicada como no robusta implica que probablemente no estemos cumpliendo algunos de los supuestos necesarios para hacer regresión, como normalidad multicolinealidad.

> Yendo a los signos de los coeficientes de la regresión, algunos de los resultados son esperables, más allá de que el modelo en general no sea adecuado, a saber:

  adFlag es positivo, habíamos visto en los desafíos anteriores que el revenue asociado a los productos publicitados era más alto que los no publicitados, lo que en principio habla de una cierta efectividad de la publicidad

  click y basket son negativos, mientras que order es positivo, dado que order indica compra, es lógico esperar una relación positiva. En cuanto a click y basket, es necesario un análisis detallado sobre que lleva a los usuarios que hacen click en un producto a no generar revenue, y ver que tan estrecha es la relación entre click, basket y order.

  price tiene un coeficiente ligeramente negativo.

In [ ]:
y_pred = modelo_ols.predict(X_test)


Generamos la tabla de comparaciones entre actual y predicted, pero dado el total de observaciones (más de 550 mil), va a ser más util graficarlo.


In [ ]:
comparacion = pd.DataFrame({'valor_real': y_test, 'valor_predicho': y_pred})
print(comparacion)

In [ ]:
sns.scatterplot(x='valor_real', y='valor_predicho', data=comparacion)
plt.xlabel('Valores reales en el subset de testing')
plt.ylabel('Valores predichos por la regresión')
plt.title('Comparación de valores reales y predichos por el modelo')
plt.show()

##**Correlación de los regresores**

En esta sección vemos que los regresores no están muy fuertemente correlacionados con la variable dependiente, como se mencionaba más arriba, lo cual es fundamental para entender la poca potencia del modelo de regresión lineal para predecir el revenue.

In [ ]:
regresores=X

correlaciones=regresores.corrwith(farmacia['revenue'])
print(correlaciones)

#**Aplicación de un modelo de árbol de decisión para predecir ventas**

Como se excplicó en trabajos y secciones anteriores, tenemos una variable llamada order que toma valores 0 (no hay venta en el registro) y 1 (la fila representa una venta).

Dado que los precios son positivos, las ventas generan revenues, entonces, vamos a tomar un enfoque indirecto, y estimar un modelo de árbol de decisión para predecir el valor de la variable order. Si conocemos los precios (y en este dataset siempre conocemos los precios), y si podemos predecir si habrá ventas, entonces estaremos en una buena posición para predecir el revenue.

##**Selección de variables explicativas y variable dependiente y creacion de test & training sets**

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.tree import plot_tree


Dividimos el dataset en variable dependiente (order) y explicativas.

In [ ]:
X = farmacia.drop(['order', 'lineID', 'group' , 'content', 'pharmForm','unit','category','campaignIndex'], axis=1) #group lo eliminamos porque ya tenemos Product ID, y group son agrupamientos de ellos. LineID es simplemente el codigo de linea

y =farmacia['order']

In [ ]:
#Nos aseguramos que nuestras variables explicativas no sean string

print(X.dtypes)

Dividimos el dataset en sets de train y test. Tomamos el 20% de la muestra para testear.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

##**Creamos y ajistamos el modelo de árbol de decisión**

In [ ]:
clf=DecisionTreeClassifier(random_state=1234, max_depth=4)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
predicciones = clf.predict(X_test)


print(accuracy_score(y_test,predicciones))

In [ ]:
comparacion = pd.DataFrame({'valor_real': y_test, 'valor_predicho': y_pred})
print(comparacion)

In [ ]:
confusion_matrix(y_test,predicciones, labels=[0,1])

In [ ]:
precision_score(y_test, predicciones)

In [ ]:
plot_tree(clf, fontsize=10)
plt.show()

Como puede verse, el modelo de árbol de decisión es relativamente eficaz a la hora de predecir la variable 'order'.